In [1]:
import urllib.request
import string
import json 
import numpy as np
from functools import reduce
from bs4 import BeautifulSoup
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from pythonds.basic.stack import Stack

#### Module 1
### Connect to the CSI courses website
url = 'https://catalogue.uottawa.ca/en/courses/csi/'
response = urllib.request.urlopen(url)
soup_packetpage = BeautifulSoup(response, 'lxml')
items = soup_packetpage.findAll("div", class_ = "courseblock")

lemmat = WordNetLemmatizer()
stemmer = PorterStemmer()

#### Module 3
def preprocessing(str_info):
    tokenized_word = word_tokenize(str_info)
    
    wordnet_lemmatizer = WordNetLemmatizer()
    porter_stemmer = PorterStemmer()

    sp_removed_word = []
    # Stopword Removal
    stop_words = set(stopwords.words('english')\
                     +stopwords.words('french')\
                     +list(string.ascii_lowercase)\
                     +[ str(i) for i in range(0,10)])
    for word in tokenized_word:
        # Lemmatization
        word = wordnet_lemmatizer.lemmatize(word)
    
        # Stemming - Porter Stemmer 
        word = porter_stemmer.stem(word)
    
        if word.lower() not in stop_words:
            sp_removed_word.append(
                word.lower())
    
    return sp_removed_word



def corpus_preprocessing(items):
    '''input a list of result.Tags and return a list of dictionarized courses'''
    # 1. Save the document ID, name(e.g., Database I) 
    # and the corresponding description into a dict - course:

    # 2. Save all courses info into a list of dict - courses
    courses = [] 
    docID = 0
    for tag in items:
        # Eliminate French Courses and the unit info
        regex = r'(?P<code>\w{3}\xa0\d{1}[1|2|3|4|9|0]\d{2})\s(?P<name>.*)'
        
        course = re.match(regex ,tag.find('strong').text.lower()) 
        if course:
            course = course.groupdict()
            course['docID'] = docID
            course['name'] = re.sub(r'\([^)]*\)', '', course['name'])
            course['description'] = tag.find('p',{'class':'courseblockdesc noindent'})
            if course['description']:
                course['description'] = course['description'].text
                courses.append(course)
            docID += 1
    return courses
# test case:
# courses is a list of dict
# keys: docID, name, and description
courses = corpus_preprocessing(items)

list

In [8]:
courses[0]

{'code': 'csi\xa01306',
 'name': 'computing concepts for business ',
 'docID': 0,
 'description': '\nIntroduction to computer-based problem solving from the perspective of the business world. Design of algorithms for solving business problems. Basics of computer programming in a modern programming language. Solving business problems using application packages including spreadsheets and databases. Basics of web design. Collaborative tools. Using open source software.'}

In [9]:
# Write the list of dictionaries into json file
import json
with open('course_dic_lst','w') as fOut:
    json.dump(courses, fOut)

In [10]:
# Read the data stored in the json file
with open('course_dic_lst') as json_file:
    data = json.load(json_file)
    for p in data:
        print(p['description'])
        break


Introduction to computer-based problem solving from the perspective of the business world. Design of algorithms for solving business problems. Basics of computer programming in a modern programming language. Solving business problems using application packages including spreadsheets and databases. Basics of web design. Collaborative tools. Using open source software.


In [3]:
def four_step_processing(curr):
    # Tokenization 
    #   Concatenate course name and the description into one string
    str_name = curr.get("name")
    str_description = curr.get("description")
    str_info = str_name + str_description

    #   Remove the punctuation from the string
    for c in string.punctuation:
        str_info = str_info.replace(c,"")
    return preprocessing(str_info)


In [25]:
def all_occurred_word(courses):
    list_of_word_lists = []
    for i in courses:
        list_of_word_lists.append(four_step_processing(i))
    
    # Combine all sublists into one list
    return list_of_word_lists ,sum(list_of_word_lists,[])

list_of_word_lists ,one_list_word = all_occurred_word(courses)



In [21]:
one_list_word[:10]

['comput',
 'concept',
 'busi',
 'introduct',
 'computerbas',
 'problem',
 'solv',
 'perspect',
 'busi',
 'world']

In [26]:
def dictionary_building(courses):
    # Indexing the word
    #   Create the transform
    vectorizer = TfidfVectorizer()

    lw, one_list_word = all_occurred_word(courses)
    
    #   Tokenize and build vocab
    bow = vectorizer.fit_transform(one_list_word) 
    #   Summarize 
    indexed_word_dict = vectorizer.vocabulary_
    print("feature names: ",vectorizer.get_feature_names())
    print("idf: ",vectorizer.idf_)
    return indexed_word_dict
# test case:
# indexed_word_dict = dictionary_building(courses)

In [14]:

dict(list(indexed_word_dict.items())[0:2])

{'comput': 233, 'concept': 236}

In [22]:
len(indexed_word_dict)

1136

In [ ]:
# Optional Module
# Spelling correction: edit distance
# ref: https://bit.ly/2T37dTt 
def compute_dist(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

def levenshteinDistance(s1):
    # find the most similar word to one_list_word
    min_dist = 1000
    closest = None
    if s1 not in one_list_word:
        for s2 in one_list_word:
            dist = compute_dist(s1, s2)
            if dist < min_dist:
                min_dist = dist
                closest = s2
    else:
        closest = None
        min_dist = -1

    return min_dist , closest

In [ ]:
#### Module 4
# arguments: courses - document collection; indexed_word_dict - dictionary
def inverted_index_construction(courses,indexed_word_dict):
    docs = [four_step_processing(i) for i in courses]
    inv_indx = defaultdict(list)
    for idx, text in enumerate(docs):
        for word in text:
            inv_indx[word].append(idx)

    result = {}
    for key, val in inv_indx.items():
        d = defaultdict(int)
        for i in val:
            d[i] += 1
        result[key] = d.items()
    
    #result format: 'word':dic_items([(docID, weight), (docID, weight), ...])
    return result
# test case:
# term_doc_ids = inverted_index_construction(courses,indexed_word_dict)

#### Module 5
def corpus_access(input_docIDs):
    output_docs = []
    length = len(input_docIDs)
    for i in range(length):
        curr_docID = input_docIDs.pop()
        output_docs.append(courses[i])
    return output_docs

#### Module 6    
def isOperator(word):
    if(word.upper() in ['AND', 'OR', 'AND_NOT', '(', ')']):
        return True
    else:
        return False

def query_processing(user_query):
    '''Query processing: 1. transform infix format to postfix format;
                            ref: https://bit.ly/28OMA5X 
                         2. deal with the wildcard '''
    # Split the query into a list of word, including the parentheses
    pattern = re.compile(r"([.()!])")
    tmp = (pattern.sub(" \\1 ", user_query)).split(" ")
    tokenList = list(filter(None,tmp))
    
    # Transform infix to postfix 
    opStack = Stack()
    postfixList = []
    prec = {}
    prec["("] = 1
    prec["AND"] = 2
    prec["OR"] = 2
    prec["AND_NOT"] = 2
    for t in tokenList:
        if (not(isOperator(t))):
            postfixList.append(t)
        elif t == "(":
            opStack.push(t)
        elif t == ")":
            topToken = opStack.pop()
            while topToken != "(":
                postfixList.append(topToken)
                topToken = opStack.pop()
        else:
            while (not opStack.isEmpty()) and  (prec[opStack.peek()] >= prec[t]):
                postfixList.append(opStack.pop())
            opStack.push(t)

    while (not opStack.isEmpty()):
        postfixList.append(opStack.pop())
    return " ".join(postfixList)

# test case:
# user_query = "algorithm AND (data OR structure) OR (comp AND ca)"
# user_query = "perspective AND business"

# postfixQuery = query_processing(user_query)
# postfixQuery1 = query_processing(user_query1)
# print(postfixQuery1)

# print(postfixQuery):
# algorithm data structure OR AND comp prog AND OR
# ["algorithm", "data", "structure", "OR", "AND  "comp", "prog", "AND", "OR"]

def get_term_docs_list(term_doc_ids,term):
    term_docs_result = []
    
    # getting all of the doc_id list of tuple
    if term not in term_doc_ids.keys():
        term = 0
    else:
        # if the given term is incorrect 
        pass        
    term_docs = term_doc_ids.get(term)
    for k , v in term_docs:
        term_docs_result.append(k)
    return term_docs_result


# Given dict_item-type args, return two lists
def two_terms_docs_lists(term_doc_ids, term1, term2):
    term1_docs_list = []
    term2_docs_list = []
    if(type(term1)!=list and type(term2)!=list):
        term1_docs_list = get_term_docs_list(term_doc_ids, term1)
        term2_docs_list = get_term_docs_list(term_doc_ids, term2)
        
    elif(type(term1)!=list):
        term1_docs_list = get_term_docs_list(term_doc_ids, term1)

    elif(type(term2)!=list):
        term2_docs_list = get_term_docs_list(term_doc_ids, term2)
        
    else: # both of them are lists
        term1_docs_list = term1
        term2_docs_list = term2

    return term1_docs_list, term2_docs_list
    

def and_op(term1_docs_list, term2_docs_list): 
    res = list(set(term1_docs_list) & set(term2_docs_list))
    return res

def or_op(term1_docs_list, term2_docs_list):
    res = list(set(term1_docs_list) | set(term2_docs_list))
    return res 

def not_op(term1_docs_list, term2_docs_list, last_dict):
    last_doc_id = last_dict.get('docID')
    all_docs = range(0, last_doc_id)
    not_t2 = [x for x in all_docs if x not in term2_docs_list]
    res = and_op(term1_docs_list, not_t2)
    return res 

def find_list_of_terms(term_doc_ids):    
    list_of_terms = []
    for k, v in term_doc_ids.items():
        list_of_terms.append(k)

    return list_of_terms

def boolean_retrieval(query, courses):
    postfixQuery = query_processing(query)
 
    indexed_word_dict = dictionary_building(courses)
    term_doc_ids = inverted_index_construction(courses,indexed_word_dict)
    
    list_of_terms = find_list_of_terms(term_doc_ids)

    wordnet_lemmatizer = WordNetLemmatizer()
    porter_stemmer = PorterStemmer()
    edit_dist = None
    # Turn the query into a list of word
    word_list = postfixQuery.split(" ")
    tmp = []
    for term in word_list:
        if (not isOperator(term)):
            # lemmatize
            lemmatized_term = wordnet_lemmatizer.lemmatize(term)
            # stemming 
            stemmed_term = porter_stemmer.stem(term)
            
            dist , fix_stemmed_term = levenshteinDistance(term)
            
            if fix_stemmed_term: 
                stemmed_term = fix_stemmed_term
                edit_dist = (dist, fix_stemmed_term)
            
            stemmed_word_doc_ids = get_term_docs_list(term_doc_ids, stemmed_term)
            
            tmp.append(stemmed_word_doc_ids)
                
        elif(isOperator(term)):
            t1 = tmp.pop()
            t2 = tmp.pop()
            l1, l2 = two_terms_docs_lists(term_doc_ids,t1,t2)
            if (term == "AND"):
                curr = and_op(l1, l2)
            elif(term == "OR"):
                curr = or_op(l1, l2)
            elif(term == "AND_NOT"):
                last_dict = courses[-1]
                curr = and_not_op(t1, t2, last_dict)
            tmp.append(curr)
    
    # ======================
    #  compute edit_dist 
    # ======================
    if edit_dist:
        origin_query = ""
        for token in word_list:
            dist = compute_dist(token,edit_dist[1])
            if dist <= edit_dist[0]:
                origin_query += edit_dist[1] + " "
            else:
                origin_query += token + " "
        
        edit_dist = origin_query
        
    
    return reduce((lambda x , y: or_op(x,y)),tmp) , edit_dist

#### Module 7
def vsm(query, courses):
    #     add:
    list_of_word_lists,one_list_word = all_occurred_word(courses)
    
    # Find idf for all word in the collection of docs
    vectorizer = TfidfVectorizer()
    bow = vectorizer.fit_transform(one_list_word) 
    
    size_of_words = len(vectorizer.vocabulary_.keys())
    matrix = np.zeros((len(courses), size_of_words))
    

    
    for document_index, words in enumerate(list_of_word_lists):
        for word in words:
            matrix[document_index, 
                    vectorizer.vocabulary_[ word.lower() ] ] += 1

    word_idf = vectorizer.idf_
    tf_idf = matrix * word_idf

    query_vector = np.zeros((size_of_words))
    query = preprocessing(query)
    
    edit_dist = None 

    for word in query:
        if word not in vectorizer.vocabulary_.keys():
            edit_dist = levenshteinDistance(word)
            query_vector[vectorizer.vocabulary_[edit_dist[1]]] += 1
        else:
            query_vector[vectorizer.vocabulary_[word]] += 1
    
    
    # ======================
    #  compute edit_dist 
    # ======================
    if edit_dist:
        origin_query = ""
        for token in query:
            dist = compute_dist(token,edit_dist[1])
            if dist == edit_dist[0]:
                origin_query += edit_dist[1] + " "
            else:
                origin_query += token + " "
        
        edit_dist = origin_query

    query_vector =  query_vector.T * word_idf
    rank = np.matmul(matrix , query_vector)
    
    return list(reversed( np.argsort(rank))) , rank , edit_dist




    
    

In [ ]:
# test case: 
# tf = vsm("algorithm", courses)

def test():
    indexed_word_dict = dictionary_building(courses)
    term_doc_ids = inverted_index_construction(courses,indexed_word_dict)
    
    input_docIDs = [10, 11, 12]
    output_docs = corpus_access(input_docIDs)
    print("corpus complete")

    boolean_test_query = "algorithm AND (data OR structure) OR (comp AND ca)"
    test_postfixQuery = boolean_retrieval(boolean_test_query, courses)
    print("boolean complete")

    vsm_test_query = "algorithm data"
    test_vsm = vsm(vsm_test_query, courses)
    print("vsm complete")
